In [157]:
import torch.optim as optim
import importlib
import REBRNN
import torch
import pickle
importlib.reload(REBRNN)

<module 'REBRNN' from '/Users/tiril/Documents/IndividualProject/nuclear_repo/knowledge_extraction/relation_extraction/REBRNN.py'>

In [162]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model for NER 
model_name = 'dslim/distilbert-NER'

# Custom keywords
try:
    with open('keyword_matching/directory.pkl', 'rb') as file:
        keywords = pickle.load(file)
except FileNotFoundError or FileExistsError:
    with open('directory.pkl', 'rb') as file:
        keywords = pickle.load(file)

# Hyperparameters
input_size = 512
num_layers = 4          # may require tuning
hidden_size = 256       # may require tuning
num_classes = 97        # 96 different relations plus '0' for no relation
learning_rate = 0.001   # may require tuning
batch_size = 64
num_epochs = 5

In [167]:
for k in keywords:
    print(k)

FUEL
FUEL_CYCLE
SMR_DESIGN
REACTOR
SMR
POLITICAL


In [164]:
import REBRNN
importlib.reload(REBRNN)

dataset='train_annotated'
input_size = 512
batch_size = 32
model_name = 'dslim/distilbert-NER'
custom_keywords = keywords

train = REBRNN.CustomDocREDDataset(
    dataset='train_annotated',
    input_size=input_size,
    batch_size=batch_size,
    model_name=model_name,
    custom_keywords=custom_keywords,
    device=device,
    shuffle=True,
    length=10
)

Preprocessing took 7.33 minutes. 3 instances (30.00%) exceeded max length.


In [ ]:
print(train.)

In [155]:
model = REBRNN.RelationExtractorBRNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    num_classes=num_classes,
    batch_size=batch_size,
    model_name=model_name,
    device=device,
    custom_keywords=keywords,
    threshold=0.7
)

In [104]:
train_raw, _, test_raw, validation_raw = model.load_data(get_distant=False)

In [135]:
length = 10
train = model.preprocessor(train_raw, length=2*length)
test = model.preprocessor(test_raw, length=length)
validation = model.preprocessor(validation_raw, length=length)

model.set_data_loaders(train, validation, test)

Preprocessing took 0.08 minutes. 7 instances (35.00%) exceeded max length.
Preprocessing took 0.04 minutes. 2 instances (20.00%) exceeded max length.
Preprocessing took 0.04 minutes. 2 instances (20.00%) exceeded max length.


In [143]:
model.set_data_loaders(train, validation, test)

In [64]:
'''
max_length=512 and length=100:
Preprocessing took 0.59 minutes. 58 instances (29.00%) exceeded max length.
Preprocessing took 0.29 minutes. 36 instances (36.00%) exceeded max length.
Preprocessing took 0.29 minutes. 33 instances (33.00%) exceeded max length.

max_length=1024 and length=100:
Preprocessing took 0.60 minutes. 1 instances (0.50%) exceeded max length.
Preprocessing took 0.29 minutes. 0 instances (0.00%) exceeded max length.
Preprocessing took 0.30 minutes. 1 instances (1.00%) exceeded max length.
'''

Preprocessing took 0.92 minutes. 58 instances (29.00%) exceeded max length.
Preprocessing took 0.50 minutes. 36 instances (36.00%) exceeded max length.
Preprocessing took 0.46 minutes. 33 instances (33.00%) exceeded max length.


'\nmax_length=512 and length=100:\nPreprocessing took 0.59 minutes. 58 instances (29.00%) exceeded max length.\nPreprocessing took 0.29 minutes. 36 instances (36.00%) exceeded max length.\nPreprocessing took 0.29 minutes. 33 instances (33.00%) exceeded max length.\n\nmax_length=1024 and length=100:\nPreprocessing took 0.60 minutes. 1 instances (0.50%) exceeded max length.\nPreprocessing took 0.29 minutes. 0 instances (0.00%) exceeded max length.\nPreprocessing took 0.30 minutes. 1 instances (1.00%) exceeded max length.\n'

In [130]:
custom_loss_fn = REBRNN.CustomLoss(threshold=0.8)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [151]:
def train_model(model, custom_loss_fn, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_train_loss = 0

        for batch in model.train_loader:
            print(batch['embeddings'].size())

        for batch in model.train_loader:
            optimizer.zero_grad()

            embeddings = batch['embeddings'].to(model.device)
            print(embeddings.size())
            entity_pairs = batch['entity_pairs']
            triplets = batch['labels']

            outputs = model(embeddings)
            loss = custom_loss_fn(entity_pairs, outputs, triplets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(model.train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in model.val_loader:
                embeddings = batch['embeddings'].to(model.device)  # Use precomputed embeddings
                entity_pairs = batch['entity_pairs']
                triplets = batch['labels']

                outputs = model(embeddings)  # Pass embeddings to the model
                loss = custom_loss_fn(entity_pairs, outputs, triplets)
                total_val_loss += loss.item()
        
        avg_val_loss = total_val_loss / len(model.val_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")
        
        model.train()

In [152]:
train_model(model, custom_loss_fn, optimizer, num_epochs=5)

torch.Size([20, 512, 768])
torch.Size([20, 512, 768])


RuntimeError: input.size(-1) must be equal to input_size. Expected 512, got 768

In [156]:
embeddings = torch.randn(20, 512, 768)
output = model.forward(embeddings)

RuntimeError: Expected hidden[0] size (8, 512, 256), got [8, 20, 256]

        batch_size = 16
        for batch_start in range(0, length, batch_size):
            batch_end = min(batch_start + batch_size, length)
            batch_data = docred_data.iloc[batch_start:batch_end]

            sents_list = []
            vertexSet_list = []
            labels_list = []

            for instance in batch_data.itertuples():
                sents, vertexSet, labels = self.get_info(instance)
                sents_list.append(sents)
                vertexSet_list.append(vertexSet)
                labels_list.append(labels)

In [374]:
import importlib
import CustomDocREDDataset
importlib.reload(CustomDocREDDataset)

dataset='train_annotated'
input_size = 512
batch_size = 32
model_name = 'dslim/distilbert-NER'
custom_keywords = keywords

train = CustomDocREDDataset.CustomDocREDDataset(
    dataset='train_annotated',
    input_size=input_size,
    batch_size=batch_size,
    model_name=model_name,
    custom_keywords=custom_keywords,
    device=device,
    shuffle=True,
    #length = 200
)

Starting preprocessing...
0.00% finished
1.64% finished
3.28% finished
4.91% finished
6.55% finished
8.19% finished
9.83% finished
11.46% finished
13.10% finished
14.74% finished
16.38% finished
18.02% finished
19.65% finished
21.29% finished
22.93% finished
24.57% finished
26.20% finished
27.84% finished
29.48% finished
31.12% finished


In [321]:
instance = train.raw_data.iloc[10]
sents, vertexSet, labels = train.get_info(instance)
entities = train.extract_entities(sents)
entity_positions = train.get_entity_positions(sents, entities)

In [330]:
for vS in vertexSet:
    print(vS)
print(labels['head'])
print(labels['tail'])
print(labels['relation_text'])

[{'name': 'Second River', 'sent_id': 0, 'pos': [1, 3], 'type': 'LOC'}, {'name': 'Second River', 'sent_id': 2, 'pos': [5, 7], 'type': 'LOC'}, {'name': 'Second River', 'sent_id': 7, 'pos': [9, 11], 'type': 'LOC'}, {'name': 'Second River', 'sent_id': 4, 'pos': [22, 24], 'type': 'LOC'}]
[{'name': 'Watsessing River', 'sent_id': 0, 'pos': [5, 7], 'type': 'LOC'}]
[{'name': 'New Jersey', 'sent_id': 0, 'pos': [12, 14], 'type': 'LOC'}]
[{'name': 'United States', 'sent_id': 0, 'pos': [16, 18], 'type': 'LOC'}]
[{'name': 'Passaic River', 'sent_id': 0, 'pos': [26, 28], 'type': 'LOC'}, {'name': 'Passaic River', 'sent_id': 6, 'pos': [5, 7], 'type': 'LOC'}]
[{'name': 'Newark Bay', 'sent_id': 0, 'pos': [36, 38], 'type': 'LOC'}, {'name': 'Newark Bay', 'sent_id': 6, 'pos': [12, 14], 'type': 'LOC'}]
[{'name': 'West Orange', 'sent_id': 1, 'pos': [4, 6], 'type': 'LOC'}, {'name': 'West Orange', 'sent_id': 2, 'pos': [1, 3], 'type': 'LOC'}]
[{'name': 'Passaic', 'sent_id': 1, 'pos': [8, 9], 'type': 'LOC'}]
[{'na

In [331]:
triplets = train.make_triplets(vertexSet, labels)
for t in triplets:
    print(t)

{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 28, 'text': 'located in the administrative territorial entity'}, 'tail': [{'s_id': 0, 'pos': [12, 14]}]}
{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 59, 'text': 'mouth of the watercourse'}, 'tail': [{'s_id': 0, 'pos': [26, 28]}, {'s_id': 6, 'pos': [5, 7]}]}
{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 59, 'text': 'mouth of the watercourse'}, 'tail': [{'s_id': 0, 'pos': [36, 38]}, {'s_id': 6, 'pos': [12, 14]}]}
{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 2, 'text': 'country'}, 'tail': [{'s_id': 0, 'pos': [16, 18]}]}
{'head': [{'s_id': 0, 'pos': [5,

In [344]:
pairs = train.make_pairs(entity_positions)
for p in pairs:
    print(p)

({'s_id': 1, 'pos': [4, 6], 'entity': 'LOC'}, {'s_id': 4, 'pos': [9, 10], 'entity': 'LOC'})
({'s_id': 1, 'pos': [8, 9], 'entity': 'LOC'}, {'s_id': 0, 'pos': [16, 18], 'entity': 'LOC'})
({'s_id': 8, 'pos': [1, 3], 'entity': 'MISC'}, {'s_id': 1, 'pos': [8, 9], 'entity': 'LOC'})
({'s_id': 0, 'pos': [1, 3], 'entity': 'LOC'}, {'s_id': 2, 'pos': [14, 15], 'entity': 'LOC'})
({'s_id': 0, 'pos': [36, 38], 'entity': 'LOC'}, {'s_id': 0, 'pos': [1, 3], 'entity': 'LOC'})
({'s_id': 0, 'pos': [1, 3], 'entity': 'LOC'}, {'s_id': 2, 'pos': [1, 3], 'entity': 'LOC'})
({'s_id': 2, 'pos': [1, 3], 'entity': 'LOC'}, {'s_id': 2, 'pos': [24, 25], 'entity': 'LOC'})
({'s_id': 2, 'pos': [26, 27], 'entity': 'LOC'}, {'s_id': 5, 'pos': [11, 12], 'entity': 'LOC'})
({'s_id': 0, 'pos': [26, 28], 'entity': 'LOC'}, {'s_id': 2, 'pos': [26, 27], 'entity': 'LOC'})
({'s_id': 0, 'pos': [12, 14], 'entity': 'LOC'}, {'s_id': 4, 'pos': [22, 24], 'entity': 'LOC'})
({'s_id': 1, 'pos': [4, 6], 'entity': 'LOC'}, {'s_id': 2, 'pos': [50

In [347]:
sents, vertexSet, labels = train.get_info(instance)
triplets = train.make_triplets(vertexSet, labels)
for t in triplets:
    print(t)

{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 28, 'text': 'located in the administrative territorial entity'}, 'tail': [{'s_id': 0, 'pos': [12, 14]}]}
{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 59, 'text': 'mouth of the watercourse'}, 'tail': [{'s_id': 0, 'pos': [26, 28]}, {'s_id': 6, 'pos': [5, 7]}]}
{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 59, 'text': 'mouth of the watercourse'}, 'tail': [{'s_id': 0, 'pos': [36, 38]}, {'s_id': 6, 'pos': [12, 14]}]}
{'head': [{'s_id': 0, 'pos': [1, 3]}, {'s_id': 2, 'pos': [5, 7]}, {'s_id': 7, 'pos': [9, 11]}, {'s_id': 4, 'pos': [22, 24]}], 'relation': {'id': 2, 'text': 'country'}, 'tail': [{'s_id': 0, 'pos': [16, 18]}]}
{'head': [{'s_id': 0, 'pos': [5,

In [281]:
def char_to_word_position(sent, start, end):
    words = sent.split()
    current_char_index = 0
    start_word_index = -1
    end_word_index = -1

    for i, word in enumerate(words):
        word_length = len(word)
        
        if current_char_index <= start < current_char_index + word_length:
            start_word_index = i
        if current_char_index < end <= current_char_index + word_length:
            end_word_index = i + 1
        
        current_char_index += word_length + 1
        if start_word_index != -1 and end_word_index != -1:
            break

    return [start_word_index, end_word_index]

print(char_to_word_position(sent, start, end))

[13, 17]


In [228]:
import pandas as pd
instance = train.raw_data.iloc[10]
sents, vertexSet, labels = train.get_info(instance)
entities = train.extract_entities(sents)
triplets = train.make_triplets(vertexSet, labels)
i = 2

for s in sents:
    print(s)

for vS in vertexSet:
    print(vS)

vertexSetDataFrame = pd.DataFrame(vertexSet)
display(vertexSetDataFrame)


The Second River , or Watsessing River , in the state of New Jersey in the United States , is the second main tributary of the Passaic River encountered while travelling upstream from its mouth at Newark Bay .
From its source in West Orange to the Passaic , it is approximately 5   mi ( 8   km ) long .
From West Orange , the Second River passes generally easterly through the towns of Orange and East Orange , where it is joined by Wigwam , Parrow , and Nishuane Brooks , then turns slightly to the north ( though still generally easterly ) , and enters the town of Bloomfield .
Here , at Watsessing Park , it is joined by Toney 's Brook .
After leaving Bloomfield , it enters the town of Belleville ( which was , early in its history , also named " Second River " ) , and turns slightly to the south ( though still generally easterly ) .
Soon into Belleville , it forms the border between Belleville and Newark .
Finally , it joins the Passaic River , on its way to Newark Bay .
In the 19th and ear

,0,1,2,3
0,"{'name': 'Second River', 'sent_id': 0, 'pos': ...","{'name': 'Second River', 'sent_id': 2, 'pos': ...","{'name': 'Second River', 'sent_id': 7, 'pos': ...","{'name': 'Second River', 'sent_id': 4, 'pos': ..."
1,"{'name': 'Watsessing River', 'sent_id': 0, 'po...",None,None,None
2,"{'name': 'New Jersey', 'sent_id': 0, 'pos': [1...",None,None,None
3,"{'name': 'United States', 'sent_id': 0, 'pos':...",None,None,None
4,"{'name': 'Passaic River', 'sent_id': 0, 'pos':...","{'name': 'Passaic River', 'sent_id': 6, 'pos':...",None,None
5,"{'name': 'Newark Bay', 'sent_id': 0, 'pos': [3...","{'name': 'Newark Bay', 'sent_id': 6, 'pos': [1...",None,None
6,"{'name': 'West Orange', 'sent_id': 1, 'pos': [...","{'name': 'West Orange', 'sent_id': 2, 'pos': [...",None,None
7,"{'name': 'Passaic', 'sent_id': 1, 'pos': [8, 9...",None,None,None
8,"{'name': '5 mi', 'sent_id': 1, 'pos': [13, 1...",None,None,None
9,"{'name': '8 km', 'sent_id': 1, 'pos': [17, 20]...",None,None,None


In [278]:
def calculate_difference(sents, vertexSet):
    gold_pos = []
    ner_pos = []

    for i, s in enumerate(sents):
        entities = train.ner_pipeline(s)
        merged = train.merge_result(entities)
        joined = train.combine_entities(merged)

        for vS in vertexSet:
            for item in vS:
                if item['sent_id'] == i and item['type'] in ['LOC', 'PER', 'ORG', 'MISC']:
                    new_gold = f"{i} {item['type']} {item['name']} {item['pos']}"
                    #print(new_gold)
                    gold_pos.append(new_gold)
        
        for j in joined:
            new_ner = f"{i} {j['entity']} {j['word']} {char_to_word_position(s, j['start'], j['end'])}"
            #print(new_ner)
            ner_pos.append(new_ner)
    
    matches = 0
    total = len(gold_pos)

    for g in gold_pos:
        for n in ner_pos:
            if g == n:
                matches += 1
    
    return matches, total

In [279]:
for i in range(4):
    total_matches = 0
    total_total = 0

    instance = train.raw_data.iloc[i]
    sents, vertexSet, labels = train.get_info(instance)
    tmp_matches, tmp_total = calculate_difference(sents, vertexSet)
    total_matches += tmp_matches
    total_total += tmp_total

print(total_matches/total_total)

0.6896551724137931


In [346]:
sent2 = ['this has three', 'this two', 'this has five whole words']

sentence_offsets = [0]
for sent in sent2:
    sentence_offsets.append(sentence_offsets[-1] + len(sent.split(' ')))

print(sentence_offsets)

[0, 3, 5, 10]
